## **Promoción de productos financieros a clientes bancarios**

En 2016, un banco minorista vendió varios productos (cuenta hipotecaria, cuenta de ahorro y cuenta de pensión) a sus clientes.
Mantuvo un registro de todos los datos históricos, y estos datos están disponibles para su análisis y reutilización.
Tras una fusión en 2017, el banco tiene nuevos clientes y quiere lanzar algunas campañas de marketing.

El presupuesto para las campañas es limitado. El banco quiere contactar a un cliente y proponerle un solo producto.

El departamento de marketing debe decidir:

-   ¿Quién debe ser contactado?
-   ¿Qué producto se debe proponer?
    Proponer demasiados productos es contraproducente, por lo que solo se propondrá un producto por contacto de cliente.
-   ¿Cómo se contactará a un cliente?
    Hay diferentes formas, con diferentes costos y eficiencia.
-   ¿Cómo pueden utilizar de manera óptima su presupuesto limitado?
-   ¿Estas campañas serán rentables?

#### **Flujo de trabajo predictivo y prescriptivo**

A partir de los datos históricos, puede entrenar un clasificador basado en productos de aprendizaje automático en el perfil del cliente (edad, ingresos, nivel de cuenta, ...) para predecir si un cliente se suscribiría a una cuenta de hipoteca, ahorro o pensión.

-   Puede aplicar este modelo predictivo a los datos de nuevos clientes para predecir qué comprará cada nuevo cliente.
-   Con estos nuevos datos, usted decide qué ofertas se proponen. Se determina qué producto se ofrece a qué cliente a través de qué canal:
    -   a. con un codicioso algoritmo que reproduce lo que haría un ser humano
    -   b. utilizando un modelo de optimización con IBM Decision Optimization.
-   Las soluciones se pueden mostrar, comparar y analizar.

**Tabla de contenido:**

-   [Comprender los datos históricos](#Comprender-los-datos-históricos)
-   [Predecir el comportamiento del cliente en 2017](#Predecir-el-comportamiento-del-cliente-en-2017)
-   [Obtenga decisiones de negocios para los datos de 2017](#Obtenga-decisiones-de-negocios-sobre-los-datos-de-2017)
-   [Conclusión sobre la toma de decisiones](#Conclusión)


In [1]:
import pandas
import plotly.express as px
import sklearn.ensemble


## **1. Comprender los datos históricos**


##### **1.1. Carga de datos clientes 2016**


In [2]:
## A cada cliente se le indica si tiene pension, ahorros o hipóteca
df_known_behaviors = pandas.read_csv("./known_behaviors.csv")
df_known_behaviors


,customer_id,age,age_youngest_child,debt_equity,gender,bad_payment,gold_card,pension_plan,household_debt_to_equity_ratio,income,...,call_center_contacts,loan_accounts,number_products,number_transactions,non_worker_percentage,white_collar_percentage,rfm_score,Mortgage,Pension,Savings
0,15,45,12,45,0,0,0,0,65,13453,...,0,4,2,1,14,19,7.602,0,0,0
1,16,43,12,43,0,0,0,0,65,13453,...,0,0,3,2,14,19,10.143,0,0,0
2,30,23,0,23,0,0,0,0,65,13453,...,0,1,0,0,14,19,0.000,0,0,0
3,42,35,8,35,1,0,0,0,65,13453,...,0,1,0,0,14,19,0.000,0,1,0
4,52,43,12,43,1,0,0,0,47,14124,...,3,1,0,0,16,35,0.000,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11018,53170,36,3,36,1,0,0,0,53,51854,...,2,0,0,0,18,27,0.000,0,1,0
11019,53173,42,9,42,0,0,0,0,55,51941,...,2,3,0,0,11,32,0.000,0,1,1
11020,53179,39,11,39,1,0,0,0,66,51858,...,0,0,0,13,11,21,19.369,0,0,1
11021,53181,55,29,55,0,0,0,0,60,51943,...,1,0,0,0,10,28,0.000,0,0,0


##### **1.2. Verificamos consumo por producto de clientes 2016**


In [3]:
n_clientes_2016 = len(df_known_behaviors)
n_hipoteca_2016 = len(df_known_behaviors[df_known_behaviors["Mortgage"] == 1])
n_pension_2016 = len(df_known_behaviors[df_known_behaviors["Pension"] == 1])
n_ahorros_2016 = len(df_known_behaviors[df_known_behaviors["Savings"] == 1])


print(f"Número de clientes: {n_clientes_2016}")
print(f"Número de clientes con hipoteca: {n_hipoteca_2016} ___ Porcentaje: {n_hipoteca_2016 / n_clientes_2016:.4%}")
print(f"Número de clientes con pension: {n_pension_2016} ___ Porcentaje: {n_pension_2016/ n_clientes_2016:.4%}")
print(f"Número de clientes con ahorros: {n_ahorros_2016} ___ Porcentaje: {n_ahorros_2016/ n_clientes_2016:.4%}")


Número de clientes: 11023
Número de clientes con hipoteca: 1664 ___ Porcentaje: 15.0957%
Número de clientes con pension: 2456 ___ Porcentaje: 22.2807%
Número de clientes con ahorros: 4222 ___ Porcentaje: 38.3017%


##### **1.3. Agregar columna de número de productos consumidos**


In [4]:
df_known_behaviors["nb_products"] = df_known_behaviors["Mortgage"] + df_known_behaviors["Pension"] + df_known_behaviors["Savings"]
df_known_behaviors.head()

,customer_id,age,age_youngest_child,debt_equity,gender,bad_payment,gold_card,pension_plan,household_debt_to_equity_ratio,income,...,loan_accounts,number_products,number_transactions,non_worker_percentage,white_collar_percentage,rfm_score,Mortgage,Pension,Savings,nb_products
0,15,45,12,45,0,0,0,0,65,13453,...,4,2,1,14,19,7.602,0,0,0,0
1,16,43,12,43,0,0,0,0,65,13453,...,0,3,2,14,19,10.143,0,0,0,0
2,30,23,0,23,0,0,0,0,65,13453,...,1,0,0,14,19,0.000,0,0,0,0
3,42,35,8,35,1,0,0,0,65,13453,...,1,0,0,14,19,0.000,0,1,0,1
4,52,43,12,43,1,0,0,0,47,14124,...,1,0,0,16,35,0.000,0,1,0,1


In [5]:
print(f"Número de clientes que tienen más de 1 producto: {len(df_known_behaviors[df_known_behaviors['nb_products'] > 1])}")
print(f"Número de clientes que tienen todos los productos: {len(df_known_behaviors[df_known_behaviors['nb_products'] == 3])}")


Número de clientes que tienen más de 1 producto: 1650
Número de clientes que tienen todos los productos: 123


##### **1.4. Gráfico de ahorro según edad e ingresos**


In [6]:
fig = px.scatter(df_known_behaviors, x="age", y="income", color="Savings", color_continuous_scale="Viridis")
fig.update_layout(
    coloraxis_showscale=False,
    title="Ahorros según edad e ingresos",
    autosize=False,
    width=500,
    height=400,
    xaxis_title="Edad",
    yaxis_title="Ingresos",
    template="ggplot2",
)
fig.show()


##### **1.5. Gráfico de hipoteca según personas en hogar y número de cuentas**


In [7]:
fig = px.scatter(df_known_behaviors, x="members_in_household", y="loan_accounts", color="Mortgage", color_continuous_scale="Viridis")
fig.update_layout(
    coloraxis_showscale=False,
    title="Hipoteca según personas en hogar y número de cuentas",
    autosize=False,
    width=600,
    height=400,
    xaxis_title="Número de personas en el hogar",
    yaxis_title="Número de cuentas",
    template="ggplot2",
)
fig.show()


##### **1.6. Gráfico de ahorros según edad e ingresos**


In [8]:
fig = px.scatter(df_known_behaviors, x="age", y="income", color="Pension", color_continuous_scale="Viridis")
fig.update_layout(
    coloraxis_showscale=False,
    title="Pension según edad e ingresos",
    autosize=False,
    width=500,
    height=400,
    xaxis_title="Edad",
    yaxis_title="Ingresos",
    template="ggplot2",
)
fig.show()


## **2. Predecir comportamiento de los clientes para 2017**


##### **2.1. Selección de columnas y de productos para entrenamiento**


In [9]:
## Selección de columnas y productos
train_columns = ["age", "income", "members_in_household", "loan_accounts"]
products = ["Savings", "Mortgage", "Pension"]

## Filtrado de columnas
X_train = df_known_behaviors[train_columns].to_numpy()

## Variables Dependientes
y_savings = df_known_behaviors["Savings"].to_numpy()
y_mortgage = df_known_behaviors["Mortgage"].to_numpy()
y_pension = df_known_behaviors["Pension"].to_numpy()


In [10]:
X_train

array([[   45, 13453,     2,     4],
       [   43, 13453,     2,     0],
       [   23, 13453,     2,     1],
       ...,
       [   39, 51858,     3,     0],
       [   55, 51943,     4,     0],
       [   44, 51893,     6,     0]], dtype=int64)

In [11]:
y_savings

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

##### **2.2. Entrenamiento supervisado de clientes de 2016**


In [12]:
## Entrenamiento Ahorros
model_clf_savings = sklearn.ensemble.GradientBoostingClassifier()
model_clf_savings.fit(X_train, y_savings)

## Entrenamiento Hipoteca
model_clf_mortgage = sklearn.ensemble.GradientBoostingClassifier()
model_clf_mortgage.fit(X_train, y_mortgage)

## Entrenamiento Pension
model_clf_pension = sklearn.ensemble.GradientBoostingClassifier()
model_clf_pension.fit(X_train, y_pension)


GradientBoostingClassifier()

##### **2.3. Predicción clientes 2017**


In [13]:
## Dataframe clientes 2017
df_unknown_behaviors = pandas.read_csv("./unknown_behaviors.csv")
df_unknown_behaviors.head()


,customer_id,age,age_youngest_child,debt_equity,gender,bad_payment,gold_card,pension_plan,household_debt_to_equity_ratio,income,members_in_household,months_current_account,months_customer,call_center_contacts,loan_accounts,number_products,number_transactions,non_worker_percentage,white_collar_percentage,rfm_score
0,44256,38,11,38,1,0,0,0,58,47958.0,4,41,48,2,1,2,1,9,31,7.949
1,46883,30,1,30,0,0,0,0,55,48606.0,2,23,36,2,4,3,2,11,33,11.437
2,32387,41,13,41,1,0,0,0,59,42152.0,4,39,48,1,0,1,1,11,29,7.462
3,25504,42,14,42,1,0,0,0,57,39788.0,3,21,24,2,3,0,0,15,27,0.000
4,35979,42,14,42,1,0,0,0,53,44365.0,6,41,48,3,2,3,1,5,41,7.806


In [14]:
## Dataframe a predecir. Clientes de 2017 con columnas de entrenamiento
df_customers = df_unknown_behaviors[["customer_id"] + train_columns].copy()
df_customers = df_customers.rename(columns={"customer_id": "id"})
df_customers


,id,age,income,members_in_household,loan_accounts
0,44256,38,47958.0,4,1
1,46883,30,48606.0,2,4
2,32387,41,42152.0,4,0
3,25504,42,39788.0,3,3
4,35979,42,44365.0,6,2
...,...,...,...,...,...
2751,40719,63,46462.0,5,0
2752,27706,42,40860.0,2,3
2753,104380,58,70944.0,5,0
2754,23228,69,38196.0,2,1


In [15]:
df_customers["Savings"] = model_clf_savings.predict(df_unknown_behaviors[train_columns].to_numpy())
df_customers["Mortgage"] = model_clf_mortgage.predict(df_unknown_behaviors[train_columns].to_numpy())
df_customers["Pension"] = model_clf_pension.predict(df_unknown_behaviors[train_columns].to_numpy())
df_customers

,id,age,income,members_in_household,loan_accounts,Savings,Mortgage,Pension
0,44256,38,47958.0,4,1,0,0,0
1,46883,30,48606.0,2,4,0,0,0
2,32387,41,42152.0,4,0,0,0,0
3,25504,42,39788.0,3,3,0,0,0
4,35979,42,44365.0,6,2,0,1,0
...,...,...,...,...,...,...,...,...
2751,40719,63,46462.0,5,0,0,0,1
2752,27706,42,40860.0,2,3,0,0,0
2753,104380,58,70944.0,5,0,1,0,0
2754,23228,69,38196.0,2,1,0,0,1


##### **2.5. Gráfico predicción de ahorro según edad e ingresos**


In [16]:
fig = px.scatter(df_customers, x="age", y="income", color="Savings", color_continuous_scale="Viridis")
fig.update_layout(
    coloraxis_showscale=False,
    title="Ahorros según edad e ingresos",
    autosize=False,
    width=500,
    height=400,
    xaxis_title="Edad",
    yaxis_title="Ingresos",
    template="ggplot2",
)
fig.show()


##### **2.6. Gráfico predicción de hipoteca según personas en hogar y número de cuentas**


In [17]:
fig = px.scatter(df_customers, x="members_in_household", y="loan_accounts", color="Mortgage", color_continuous_scale="Viridis")
fig.update_layout(
    coloraxis_showscale=False,
    title="Hipoteca según personas en hogar y número de cuentas",
    autosize=False,
    width=500,
    height=400,
    xaxis_title="Edad",
    yaxis_title="Ingresos",
    template="ggplot2",
)
fig.show()


##### **2.7. Gráfico de ahorros según edad e ingresos**


In [18]:
fig = px.scatter(df_customers, x="age", y="income", color="Pension", color_continuous_scale="Viridis")
fig.update_layout(
    coloraxis_showscale=False,
    title="Ahorros según edad e ingresos",
    autosize=False,
    width=500,
    height=400,
    xaxis_title="Edad",
    yaxis_title="Ingresos",
    template="ggplot2",
)
fig.show()


#### **2.8. Porcentaje de predicciones por producto**


In [19]:
n_clientes_2017 = len(df_customers)
n_hipoteca_2017 = len(df_customers[df_customers["Mortgage"] == 1])
n_pension_2017 = len(df_customers[df_customers["Pension"] == 1])
n_ahorros_2017 = len(df_customers[df_customers["Savings"] == 1])

print(f"Número de clientes: {n_clientes_2017}")
print(f"Número de clientes con hipoteca: {n_hipoteca_2017} ___ Porcentaje: {n_hipoteca_2017 / n_clientes_2017:.4%}")
print(f"Número de clientes con pension: {n_pension_2017} ___ Porcentaje: {n_pension_2017 / n_clientes_2017:.4%}")
print(f"Número de clientes con ahorros: {n_ahorros_2017} ___ Porcentaje: {n_ahorros_2017 / n_clientes_2017:.4%}")


Número de clientes: 2756
Número de clientes con hipoteca: 380 ___ Porcentaje: 13.7881%
Número de clientes con pension: 142 ___ Porcentaje: 5.1524%
Número de clientes con ahorros: 713 ___ Porcentaje: 25.8708%


#### **2.9. Agregar columna de número de productos consumidos**


In [20]:
df_customers["nb_products"] = df_customers["Mortgage"] + df_customers["Pension"] + df_customers["Savings"]
df_customers.head()


,id,age,income,members_in_household,loan_accounts,Savings,Mortgage,Pension,nb_products
0,44256,38,47958.0,4,1,0,0,0,0
1,46883,30,48606.0,2,4,0,0,0,0
2,32387,41,42152.0,4,0,0,0,0,0
3,25504,42,39788.0,3,3,0,0,0,0
4,35979,42,44365.0,6,2,0,1,0,1


In [21]:
print(f"Número de clientes que tienen más de 1 producto: {len(df_customers[df_customers['nb_products'] > 1])}")
print(f"Número de clientes que tienen todos los productos: {len(df_customers[df_customers['nb_products'] == 3])}")


Número de clientes que tienen más de 1 producto: 112
Número de clientes que tienen todos los productos: 0


## **Nuestro Problema**

El objetivo es contactar a los clientes para venderles un solo producto, por lo que no puedes seleccionarlos todos. Sabiendo que:

-   Tenemos una predicción de quién comprará qué en la lista de nuevos clientes.
-   Sin embargo, no tenemo el presupuesto para contactarlos a todos. Tenemos varios canales de contacto con diferentes costos y efectividad.
-   Además, si se pone en contacto con un cliente, desea proponer solo un producto por cliente.

Esto aumenta la complejidad del problema: debe determinar el mejor canal de contacto, pero también debe seleccionar qué producto se venderá a un cliente determinado.
Puede ser difícil calcular esto. Para comprobarlo, utilizaremos dos técnicas:

-   Un algoritmo greedy
-   Un modelo de optimización con cplex.


## **3. Decisiones comerciales para los datos de 2017**


In [22]:
CUSTOMERS = list(df_customers.index)
PRODUCTS = ["Savings", "Mortgage", "Pension"]
CHANNELS = ["gift", "newsletter", "seminar"]

In [23]:
## Cuántos ingresos se obtienen al vender cada producto
VALUE_PER_PRODUCT = {"Savings": 200, "Mortgage": 300, "Pension": 400}

## Presupuesto total disponible
AVAILABLE_BUDGET = 25000

## Para cada canal, coste de realizar una acción de marketing y factor de éxito
COST_CHANNELS = {"gift": 20, "newsletter": 15, "seminar": 23}
FACTOR_CHANNELS = {"gift": 0.2, "newsletter": 0.05, "seminar": 0.3}

## 1 Si a un cliente se le asigno un producto. 0 dlc
ALPHA = df_customers[["Savings", "Mortgage", "Pension"]].to_dict("index")


##### **3.1. Resolvemos el problema con algoritmo greedy**

-   Formularemos un algoritmo personalizado que garantiza que el 10 % de las ofertas se **realizan** por canal eligiendo las más prometedoras por canal. Luego, el algoritmo continúa agregando ofertas hasta que se alcanza el presupuesto.


In [24]:
df_greedy_solution = pandas.DataFrame()
df_greedy_solution.index = df_customers.index
df_greedy_solution["Savings"] = 0
df_greedy_solution["Mortgage"] = 0
df_greedy_solution["Pension"] = 0

budget = 0
revenue = 0

noffers = len(CUSTOMERS)

# Asegurar el 10% por canal eligiendo el más prometedor por canal
for channel in CHANNELS:
    i = 0
    while i < (noffers // 10):
        # Encontrar una posible oferta en este canal para un cliente aún no realizada
        added = False
        for customer in CUSTOMERS:
            already = False
            for product in products:
                if df_greedy_solution.loc[customer, product] == 1:
                    already = True
                    break
            if already:
                continue
            possible = False
            possibleProduct = None
            for product in products:
                if ALPHA[customer][product] == 1:
                    possible = True
                    possibleProduct = product
                    break
            if not possible:
                continue
            df_greedy_solution.loc[customer, possibleProduct] = 1

            i = i + 1
            added = True
            budget = budget + COST_CHANNELS[channel]
            revenue = revenue + FACTOR_CHANNELS[channel] * VALUE_PER_PRODUCT[product]

            break
        if not added:
            print("NOT FEASIBLE")
            break


In [25]:
# Agregue más para completar el presupuesto
while True:
    added = False
    for channel in CHANNELS:
        if budget + COST_CHANNELS[channel] > AVAILABLE_BUDGET:
            continue
        ## Encontrar una posible oferta en este canal para un cliente aún no realizada
        for customer in CUSTOMERS:
            already = False
            for product in products:
                if df_greedy_solution.loc[customer, product] == 1:
                    already = True
                    break
            if already:
                continue
            possible = False
            possibleProduct = None
            for product in products:
                if ALPHA[customer][product] == 1:
                    possible = True
                    possibleProduct = product
                    break
            if not possible:
                continue

            # print(f"Assigning customer {df_customers.loc[customer, 'id']} with product {product} and channel {channel['name']}")
            # print("Assigning customer ", df_customers.get_value(index=customer, col="id"), " with product ", product, " and channel ", channel['name'])
            df_greedy_solution.loc[customer, possibleProduct] = 1

            i = i + 1
            added = True
            budget += COST_CHANNELS[channel]
            revenue += FACTOR_CHANNELS[channel] * VALUE_PER_PRODUCT[product]
            break
    if not added:
        print("FINISH BUDGET")
        break


FINISH BUDGET


In [26]:
df_greedy_solution


,Savings,Mortgage,Pension
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,1,0
...,...,...,...
2751,0,0,1
2752,0,0,0
2753,1,0,0
2754,0,0,1


In [27]:
n_mortgage_greedy = len(df_greedy_solution[df_greedy_solution["Mortgage"] == 1])
n_pension_greedy = len(df_greedy_solution[df_greedy_solution["Pension"] == 1])
n_savings_greedy = len(df_greedy_solution[df_greedy_solution["Savings"] == 1])

n_promotion_greedy = len(df_greedy_solution[(df_greedy_solution["Mortgage"] == 1) | (df_greedy_solution["Pension"] == 1) | (df_greedy_solution["Savings"] == 1)])

print(f"Número de cliente al que le asignpe promoción: {n_promotion_greedy}")
print(f"Número de clientes a los que se le ofrece hipoteca: {n_mortgage_greedy}")
print(f"Número de clientes a los que se le ofrece pension: {n_pension_greedy}")
print(f"Número de clientes a los que se le ofrece ahorros: {n_savings_greedy}")
print(f"Total de presupuesto gastado: {budget}")
print(f"Total de ingresos: {revenue}")


Número de cliente al que le asignpe promoción: 1123
Número de clientes a los que se le ofrece hipoteca: 299
Número de clientes a los que se le ofrece pension: 111
Número de clientes a los que se le ofrece ahorros: 713
Total de presupuesto gastado: 21712
Total de ingresos: 50800.0


#### **3.2. Modelo de optimización**


In [28]:
from docplex.mp.model import Model


In [29]:
model = Model(name="marketing_campaign", checker="on")


##### **3.2.1. Variables**


In [30]:
## Si le propongo un producto a un cliente en cierto canal
x = model.binary_var_dict([(customer, product, channel) for customer in CUSTOMERS for product in PRODUCTS for channel in CHANNELS], name="x")


##### **3.2.2. Función Objetivo**


$$ \text{maximizar } \sum_{cust \in CUST}\sum_{prod\in PROD}\sum_{cha\in CHA} \mathcal{Beneficio}_{prod} \times \mathcal{Factor}_{cha} \times \alpha_{cust,prod} \times x_{cust,prod,cha} $$


In [31]:
obj = model.sum(
    x[customer, product, channel] * FACTOR_CHANNELS[channel] * VALUE_PER_PRODUCT[product] * ALPHA[customer][product] for customer in CUSTOMERS for product in PRODUCTS for channel in CHANNELS
)

model.maximize(obj)


##### **3.2.3. Restricciones**


$$ \forall cust \in CUST:\sum_{prod\in PROD}\sum_{cha\in CHA}x_{cust,prod,cha}\le 1 $$


In [32]:
## Como máximo se ofrece 1 producto a cada cliente
for customer in CUSTOMERS:
    model.add_constraint(model.sum(x[customer, product, channel] for product in PRODUCTS for channel in CHANNELS) <= 1)


$$ \sum_{cust \in CUST}\sum_{prod\in PROD}\sum_{cha\in CHA}\mathcal{CostoProm}_{cha} \times x_{cust,prod,cha}\le \mathcal{Presupuesto} $$


In [33]:
# No exceder mi presupuesto
model.add_constraint(
    model.sum(x[customer, product, channel] * COST_CHANNELS[channel] for customer in CUSTOMERS for product in PRODUCTS for channel in CHANNELS) <= AVAILABLE_BUDGET,
    "budget",
)

print("-")


-


$$ \forall cha\in CHA:\sum_{prod\in PROD}\sum_{cust \in CUST}x_{cust,prod,cha}\le \left\lfloor \frac{\left| CUST \right|}{10}\right\rfloor $$


In [34]:
## Al menos 10% de ofertas por canal
for channel in CHANNELS:
    model.add_constraint(model.sum(x[customer, product, channel] for product in PRODUCTS for customer in CUSTOMERS) >= len(df_customers) // 10)


#### **3.3. Resolver modelo**


In [35]:
model.set_time_limit(30)
s = model.solve()


In [36]:
print(model.get_solve_status())
print(model.get_solve_details())


JobSolveStatus.OPTIMAL_SOLUTION
status  = integer optimal solution
time    = 0.344 s.
problem = MILP
gap     = 0%



In [37]:
totaloffers = model.sum(x[customer, product, channel] for customer in CUSTOMERS for product in PRODUCTS for channel in CHANNELS)
model.add_kpi(totaloffers, "nb_clients")

budgetSpent = model.sum(x[customer, product, channel] * COST_CHANNELS[channel] for customer in CUSTOMERS for product in PRODUCTS for channel in CHANNELS)
model.add_kpi(budgetSpent, "budgetSpent")

for channel in CHANNELS:
    kpi = model.sum(x[customer, product, channel] for product in PRODUCTS for customer in CUSTOMERS)
    model.add_kpi(kpi, channel)

for product in PRODUCTS:
    kpi = model.sum(x[customer, product, channel] for channel in CHANNELS for customer in CUSTOMERS)
    model.add_kpi(kpi, product)


In [38]:
model.report()


* model marketing_campaign solved with objective = 72620.000
*  KPI: nb_clients  = 1218.000
*  KPI: budgetSpent = 24989.000
*  KPI: gift        = 275.000
*  KPI: newsletter  = 275.000
*  KPI: seminar     = 668.000
*  KPI: Savings     = 690.000
*  KPI: Mortgage    = 381.000
*  KPI: Pension     = 147.000


#### **4. Conclusión**


##### **4.1. Comparación de algoritmos**

Aquí están los resultados de los 2 algoritmos:

| Algorithm | Revenue | Number of clients | Mortgage offers | Pension offers | Savings offers | Budget Spent |
| --------- | ------- | ----------------- | --------------- | -------------- | -------------- | ------------ |
| Greedy    | 50800   | 1123              | 299             | 111            | 713            | 21700        |
| CPLEX     | 72620   | 1218              | 381             | 144            | 693            | 24989        |

-   Como puede ver, con Decision Optimization, puede usar esta campaña de marketing de manera segura para contactar a <b>1218 clientes</b> de los 2756 clientes.
-   Esto conducirá a un ingreso de <b>\$72.6K</b>, significativamente mayor que el ingreso de \$50.8K proporcionado por un algoritmo codicioso.
-   Con un algoritmo codicioso, podrás:
    -   ser incapaz de concentrarse en los clientes correctos (seleccionará menos de ellos),
    -   gaste menos del presupuesto disponible para obtener menores ingresos.
    -   concéntrese en vender cuentas de ahorro que tengan los mayores ingresos


#### **5. ¿Qué pasa si aumenta nuestro presupuesto**


##### **5.1. Obtener restricción de presupuesto**


In [39]:
ct = model.get_constraint_by_name("budget")

##### **5.2. Iteración sobre presupuesto**


In [40]:
res = []
for i in range(20):
    ct.rhs = AVAILABLE_BUDGET + 1000 * i
    s = model.solve()
    assert s, "No Solution !!!"
    res.append(
        (
            AVAILABLE_BUDGET + 1000 * i,
            model.objective_value,
            model.kpi_value_by_name("nb_clients"),
            model.kpi_value_by_name("budgetSpent"),
        )
    )


In [41]:
pandas.DataFrame(res, columns=["budget", "revenue", "nb_offers", "budgetSpent"])


,budget,revenue,nb_offers,budgetSpent
0,25000,72620.0,1218.0,24989.0
1,26000,74800.0,1262.0,25998.0
2,27000,76970.0,1305.0,26990.0
3,28000,79150.0,1349.0,27999.0
4,29000,81320.0,1392.0,28991.0
5,30000,82360.0,1436.0,30000.0
6,31000,83240.0,1479.0,30992.0
7,32000,84100.0,1522.0,31981.0
8,33000,84980.0,1566.0,32993.0
9,34000,85840.0,1609.0,33982.0
